In [ ]:
import wandb
from wandb.integration.keras import WandbMetricsLogger

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
# Sweep config for selecting experiment
sweep_config = {
    'method': 'grid',
    'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
    'parameters': {
        'batch_size': {'values': [8]},
        'learning_rate': {'values': [0.0001]},
        'hidden_nodes': {'values': [128]},
        'img_size': {'values': [16]},
        'epochs': {'values': [10]},
        'experiment': {'values': ['dropout_only', 'batchnorm_only', 'full']}
    }
}

sweep_id = wandb.sweep(sweep_config, project="5-flowers-experiments")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: wsvlncyt
Sweep URL: https://wandb.ai/pritkudale-vizuara/5-flowers-experiments/sweeps/wsvlncyt


In [ ]:
def train():
    with wandb.init() as run:
        config = wandb.config

        IMG_HEIGHT = config.img_size
        IMG_WIDTH = config.img_size
        IMG_CHANNELS = 3
        CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

        def read_and_decode(filename, resize_dims):
            img_bytes = tf.io.read_file(filename)
            img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
            img = tf.image.convert_image_dtype(img, tf.float32)
            img = tf.image.resize(img, resize_dims)
            return img

        def parse_csvline(csv_line):
            record_default = ["", ""]
            filename, label_string = tf.io.decode_csv(csv_line, record_default)
            img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
            label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]
            return img, label

        # Datasets
        train_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        eval_dataset = (
            tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
            .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(config.batch_size)
            .prefetch(tf.data.AUTOTUNE)
        )

        # Build model
        model = keras.Sequential()
        model.add(keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)))

        if config.experiment == 'batchnorm_only':
            model.add(keras.layers.Dense(config.hidden_nodes, use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))

        elif config.experiment == 'dropout_only':
            model.add(keras.layers.Dense(config.hidden_nodes, activation="relu"))
            model.add(keras.layers.Dropout(0.5))

        elif config.experiment == 'full':
            model.add(keras.layers.Dense(config.hidden_nodes,
                                          kernel_regularizer=keras.regularizers.l2(0.01),
                                          use_bias=False))
            model.add(keras.layers.BatchNormalization())
            model.add(keras.layers.Activation("relu"))
            model.add(keras.layers.Dropout(0.5))

        model.add(keras.layers.Dense(len(CLASS_NAMES), activation="softmax"))

        # Compile
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=config.learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
            metrics=["accuracy"]
        )

        # Train
        callbacks = [WandbMetricsLogger(log_freq=5)]
        if config.experiment == 'full':
            callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True))

        model.fit(
            train_dataset,
            validation_data=eval_dataset,
            epochs=config.epochs,
            callbacks=callbacks
        )

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 7zn5m4dj with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: dropout_only
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: vizuara-info (pritkudale-vizuara) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
    413/Unknown 186s 442ms/step - accuracy: 0.2629 - loss: 1.6553

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 207s 494ms/step - accuracy: 0.2630 - loss: 1.6552 - val_accuracy: 0.3973 - val_loss: 1.4265
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 255s 483ms/step - accuracy: 0.3627 - loss: 1.4486 - val_accuracy: 0.4189 - val_loss: 1.3531
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 139s 330ms/step - accuracy: 0.3986 - loss: 1.3863 - val_accuracy: 0.4243 - val_loss: 1.3153
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 115s 265ms/step - accuracy: 0.4268 - loss: 1.3479 - val_accuracy: 0.4216 - val_loss: 1.2856
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 263ms/step - accuracy: 0.4176 - loss: 1.3222 - val_accuracy: 0.4405 - val_loss: 1.2799
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 143s 265ms/step - accuracy: 0.4488 - loss: 1.3061 - val_accuracy: 0.4514 - val_loss: 1.2603
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 131s 238ms/step - accuracy: 0.4601 - loss: 1.2682 - val_accuracy: 0.4703 - val_loss: 1.2393
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 237ms/step - accuracy: 0.4718 - loss: 1.25

...

batch/accuracy,▁▂▂▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███▇██
batch/batch_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,███▇▄▅▅▅▄▄▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▃▄▅▅▆▇▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▃▃▂▂▁▁
epoch/val_accuracy,▁▅▅▆▆▆▇██▆
epoch/val_loss,█▃▁▁▂▁▂▂▃▄
batch/accuracy,0.78133


wandb: Agent Starting Run: wiity0xq with config:
wandb: 	batch_size: 8
wandb: 	epochs: 10
wandb: 	experiment: full
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.0001


Epoch 1/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 95s 225ms/step - accuracy: 0.2408 - loss: 4.1889 - val_accuracy: 0.3919 - val_loss: 3.4887
Epoch 2/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 93s 225ms/step - accuracy: 0.3459 - loss: 3.6835 - val_accuracy: 0.4297 - val_loss: 3.2795
Epoch 3/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 140s 220ms/step - accuracy: 0.4029 - loss: 3.4019 - val_accuracy: 0.4676 - val_loss: 3.1398
Epoch 4/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 220ms/step - accuracy: 0.4376 - loss: 3.2195 - val_accuracy: 0.4595 - val_loss: 3.0251
Epoch 5/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 141s 341ms/step - accuracy: 0.4425 - loss: 3.0687 - val_accuracy: 0.4919 - val_loss: 2.9009
Epoch 6/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 341ms/step - accuracy: 0.4867 - loss: 2.8912 - val_accuracy: 0.4703 - val_loss: 2.8222
Epoch 7/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 145s 348ms/step - accuracy: 0.5025 - loss: 2.7646 - val_accuracy: 0.4622 - val_loss: 2.7452
Epoch 8/10
413/413 ━━━━━━━━━━━━━━━━━━━━ 200s 343ms/step - accuracy: 0.4921 - l

batch/accuracy,▁▁▂▂▂▄▄▄▅▅▅▅▆▆▆▆▆▅▆▆▇▇▇▇▇▇▇▇▅▇▇▇▇▇▇▇▇▇██
batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,██▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▃▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
epoch/accuracy,▁▃▅▅▆▇▇▇██
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▅▄▄▃▂▂▁▁
epoch/val_accuracy,▁▄▆▆█▆▆██▆
epoch/val_loss,█▆▅▅▄▃▃▂▁▁
batch/accuracy,0.53923


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
